In [1]:
from pyspark.sql.functions import *

In [2]:
topic_read = spark.read\
    .format("kafka")\
    .option("kafka.bootstrap.servers","kafka:9092")\
    .option("subscribe","topic-kvspark")\
    .load()

In [3]:
topic_read.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [13]:
topic_string = topic_read.select(col("key").cast("string"),col("value").cast("string"),"partition",
                                "timestamp")
print(topic_string.show())

+---+---------------+---------+--------------------+
|key|          value|partition|           timestamp|
+---+---------------+---------+--------------------+
|  1|Carlos 1 : 1000|        1|2021-07-06 16:30:...|
|  1|Carlos 2 : 2000|        1|2021-07-06 16:30:...|
|  1|Carlos 3 : 3000|        1|2021-07-06 16:30:...|
|  1|CArlos 4 : 4000|        1|2021-07-06 16:31:...|
|  2|  ACM 1 : 10000|        0|2021-07-06 16:34:...|
|  2|  ACM 2 : 20000|        0|2021-07-06 16:34:...|
|  2|  ACM 3 : 30000|        0|2021-07-06 16:34:...|
|  2|  ACM 4 : 40000|        0|2021-07-06 16:34:...|
+---+---------------+---------+--------------------+

None


In [5]:
topic_read_stream = spark.readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers","kafka:9092")\
    .option("subscribe","topic-kvspark")\
    .option("startingOffsets","earliest")\
    .load()

In [6]:
topic_read_stream.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
topic_string_stream = topic_read_stream.select(col("key").cast("string"),col("value").cast("string"))

In [8]:
topic_string_stream_output = topic_string_stream.writeStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers","kafka:9092")\
    .option("topic","topic-kvspark-output")\
    .option("checkpointLocation","/usr/adriano/kafka_checkpoint_kv_5")\
    .trigger(continuous="5 second")\
    .start()

In [9]:
topic_string_stream_output.status

{'message': 'Initializing sources',
 'isDataAvailable': False,
 'isTriggerActive': True}

In [10]:
topic_read2 = spark.read\
    .format("kafka")\
    .option("kafka.bootstrap.servers","kafka:9092")\
    .option("subscribe","topic-kvspark-output")\
    .load()

In [12]:
topic_string2 = topic_read2.select(col("key").cast("string"),
                                   col("value").cast("string"))
topic_string2.show()

+---+---------------+
|key|          value|
+---+---------------+
|  1|Carlos 1 : 1000|
|  1|Carlos 2 : 2000|
|  1|Carlos 3 : 3000|
|  1|CArlos 4 : 4000|
|  2|  ACM 1 : 10000|
|  2|  ACM 2 : 20000|
|  2|  ACM 3 : 30000|
|  2|  ACM 4 : 40000|
+---+---------------+

